## --- Day 8: Handheld Halting ---

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).
* acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
* jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
* nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.

For example, consider the following program:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
These instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
```

First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

In [1]:
file = open("day8.txt", "r")

lines = file.readlines()
lines = [i.replace("\n", "") for i in lines]
lines = [i.replace("+", "") for i in lines]

In [2]:
print(lines[0:10])

['nop 456', 'nop 38', 'acc 9', 'jmp 153', 'acc 15', 'nop 560', 'jmp 452', 'acc 26', 'acc 42', 'jmp 376']


In [3]:
# whenever a command is executed add 1 to its place on the list
# when a command == 2 stop and report the acc value
cmd_list = [0] * len(lines)

i = 0
acc = 0

while 2 not in cmd_list:
    # add current cmd to the cmd_list
    cmd_list[i] += 1
    cmd = lines[i].split()
    
    if cmd[0] == "nop":
        i += 1
    elif cmd[0] == "acc":
        i += 1
        # double check this isnt the 2nd run for the cmd
        if 2 not in cmd_list:
            acc += int(cmd[1])
    elif cmd[0] == "jmp":
        i += int(cmd[1])
        
print(acc)

1087


## --- Part Two ---

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:
```
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
```
If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:
```
nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
```
After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

In [4]:
# whenever a command is executed add 1 to its place on the list
# when a command == 2 stop and report the acc value
cmd_list = ([0] * len(lines))
nop_list_try = []
jmp_list_try = []
nop_list_failed = []
jmp_list_failed = []

i = 0
j = 0
acc = 0



# adding that the program should exit when i is greater than the length of lines
while i < len(lines):
    #print(acc)
    # start with testing the nop list
    if len(nop_list_try) > 0:
        j = int(nop_list_try.pop())
        nop_list_failed.append(j)
        
    elif len(jmp_list_try) > 0:
        j = int(jmp_list_try.pop())
        jmp_list_failed.append(j)
    
    # reset counters
    cmd_list = ([0] * len(lines))
    acc = 0
    i = 0
    
    while 2 not in cmd_list:
        # add current cmd to the cmd_list
        cmd_list[i] += 1
        cmd = lines[i].split()
    
        if cmd[0] == "nop":
            # this means it should be a jmp instead
            if j == i and j != 0:
                i += int(cmd[1])
            # perform normal nop
            else:
                if int(cmd[1]) != 0 and i not in nop_list_failed:
                    nop_list_try.append(i)
                i += 1
        elif cmd[0] == "acc":
            i += 1
            # double check this isnt the 2nd run for the cmd
            if 2 not in cmd_list:
                acc += int(cmd[1])
        elif cmd[0] == "jmp":
            if j == i and j != 0:
                i += 1
            else:
                if i not in jmp_list_failed:
                    jmp_list_try.append(i)
                i += int(cmd[1])

IndexError: list index out of range

In [5]:
print(acc)

780


In [6]:
print(len(nop_list_try))
print(len(nop_list_failed))

3
609


In [7]:
print(len(jmp_list_try))
print(len(jmp_list_failed))

30343
64
